# core

> Fill in a module description here

In [82]:
#| default_exp core

In [83]:
#| hide
from nbdev.showdoc import *

In [ ]:
#| export
def foo(): pass

Previous code to be migrated:

### Design considerations:

0. Should be easy for anyone to contribute new material, by adding to a python module.

1. Want to enable the user to be able to browse-via-autocomplete over canned contexts by name.

2. Want this autocomplete to work immediately, without the user needing to do anything besides importing the library.

3. Want "lazy fetching" (i.e. evaluation), so merely importing the library does not trigger dozens of network fetches.

4. nice-to-have. a facility for information _about_ the context besides just the name of it. Ideally interoperating with the normal docstring mechanism?

### Findings:

1. Existing solution uses function application to defer loading, but that also blocks providing immediate autocomplete over subtopics.

2. `AttrDict` out-of-the-box produces too many completions for a great UX but maybe goodenough

3. 

### Tentative design

- top-level module namespace defines variables `ctx_fastcore`, `ctx_docker`, etc.., which provide context on major topics like fastcore, docker, etc..
- attributes on these objects will autocomplete over sub-topics within each topic
- each subtopic is represented by an object, which defines `.get()` which will return the context information.
- A repr

- "anything returned by .get() can be fed to an llm"
- ". TAB always lists only what's relevant" (no junk in the autocomplete)

In [84]:
from typing import Callable
from ContextKit import *
from fastcore.utils import *

In [159]:
class ST:
    def __init__(self,url,getter:Callable=read_url,docs:str|None=None):
        self.url = url
        self.getter = getter
        self.docs=docs
    def get(self):
        return self.getter(self.url)
    def _children(self):
        return sorted(list(set(c for c in dir(self) if not c.startswith('_')) - {'get','getter','docs','url'}))
    def __repr__(self):
        s = ''
        if self.url: s += f"URL: {self.url}\n"
        if self.docs: s += "doc: {self.docs}\n"
        if (cs := self._children()): s += f"sub-topics: {cs}\n"            
        return s


In [169]:
class Topic:
    def _children(self):
        return sorted(list(set(c for c in dir(self) if not c.startswith('_')) - {'get','getter','docs','url'}))
    def __repr__(self):
        s = ''
        if self.__doc__: s += f"doc: {self.__doc__}\n"
        if (cs := self._children()): s += f"sub-topics: {cs}\n"            
        return s


In [170]:
class CTX_Fastlite_Sqlutils(Topic):
    "LLM contexts on fastlite-sqlutils"
    def __init__(self):
        self.fastlite_index    = ST('https://answerdotai.github.io/fastlite/index.html.md')
        self.fastlite_core     = ST('https://answerdotai.github.io/fastlite/core.html.md')
        self.sqlite_utils_docs = ST('https://sqlite-utils.datasette.io/en/stable/python-api.html',
                                    lambda url:read_url(url,sel='#sqlite-utils-python-library'))


In [171]:
ctx_fastlite_sqlutils = CTX_Fastlite_Sqlutils()

In [182]:
ctx_fastlite_sqlutils

doc: LLM contexts on fastlite-sqlutils
sub-topics: ['fastlite_core', 'fastlite_index', 'sqlite_utils_docs']

In [183]:
ctx_fastlite_sqlutils.fastlite_core

URL: https://answerdotai.github.io/fastlite/core.html.md

In [174]:
class CTX_Claudette(Topic):
    def __init__(self):
        self.async_docs    = ST('https://claudette.answer.ai/async.html.md')
        self.core_docs     = ST('https://claudette.answer.ai/core.html.md')
        self.toolloop_docs = ST('https://claudette.answer.ai/toolloop.html.md')



In [175]:
class CTX_Docker(Topic):
    def __init__(self):
        self.compose_file       = ST('https://docs.docker.com/reference/compose-file/')
        self.docker_cli         = ST('https://docs.docker.com/reference/cli/docker/')
        self.docker_compose_cli = ST('https://docs.docker.com/reference/cli/docker/compose/')
        self.dockerd            = ST('https://docs.docker.com/reference/cli/dockerd/')
        
        # page-based URLs
        url = 'https://docker-py.readthedocs.io/en/stable/{}.html'
        page_names = ['client','configs','containers','images','networks','nodes',
                 'plugins','secrets','services','swarm','volumes','api','tls',]
        pageurls = [(f"pg_{p}",url.format(p)) for p in page_names]
        for name,url in pageurls: setattr(self,name,ST(url))
        

In [176]:
class CTX_FastHtml(Topic):
    "LLM-friendly documentation for FastHtml"
    def __init__(self):
        self.fasthtml_llms_ctx = ST('https://docs.fastht.ml/llms-ctx.txt')
        self.hypermedia_summary = ST('https://gist.github.com/jph00/4ad7d35ad79013aded41b5ba535a12a3',read_gist)


In [177]:
class CTX_NBClassicServer(Topic):
    def __init__(self):
        self.nbclassic_server = ST('https://nbclassic.readthedocs.io/en/latest/nbclassic.html',
                                   lambda url:read_url(url,self='bd-article'))


In [178]:
class CTX_Fastcore(Topic):
    def __init__(self):
        self.fc_llms_ctx   = SubTopic('https://fastcore.fast.ai/llms-ctx.txt')

In [179]:
ctx_docker = CTX_Docker()
ctx_fastcore = CTX_Fastcore()
ctx_nbclassic_server = CTX_NBClassicServer()
ctx_fasthtml = CTX_FastHtml()
ctx_claudette = CTX_Claudette()
ctx_fastlite_sqlutils = CTX_Fastlite_Sqlutils()



In [181]:
ctx_docker.compose_file

URL: https://docs.docker.com/reference/compose-file/

In [91]:
ctx_fasthtml.fasthtml_llms_ctx?

In [60]:
ctx_docker.compose_file

https://docs.docker.com/reference/compose-file/

In [38]:
ctx_claudette = CTXClaudette()
ctx_fastcore = CTXFastcore()

In [39]:
ctx_fastcore.fc_llms_ctx
ctx_

https://fastcore.fast.ai/llms-ctx.txt

In [ ]:
cc2.async_docs.doc()
cc2.async_docs.asyncio.url()
ccs.async_docs.get()

In [ ]:
from contextpack import *

ctx_TOPIC.get()
ctx_TOPIC.SUBTOPIC.get()

$`ctx_TOPIC.get()`

In [ ]:
class CTRDocker:
    def __init__(self):
        self.d = {}
        self.cli 
    

In [ ]:
a

class TopicBrowser:
    def __init__(self,d:dict[str,str]):
        self.d = d
        # dynamically define attributes based on keys.

In [10]:
cc = ctx_claudette()

In [ ]:
cc.

In [ ]:
#| hide
import nbdev; nbdev.nbdev_export()